In [58]:
import astropy.table as tb
from astropy.time import Time


In [5]:
targets = tb.Table.read("../../Observation Planner/targets.csv")
targets

id,target_name,target_source,target_type,ra,dec,pmra,pmdec,parallax,Vmag,Teff,ephem_member,ephem_period,ephem_t0_a,ephem_duration_a,ephem_depth_a,ephem_t0_b,ephem_duration_b,ephem_depth_b
int32,str14,str28,str6,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64
3366,TIC 270360534,Kostov 2023 arXiv:2309.14200,QuadEB,263.430502039255,43.860752149892,-11.9616,-23.9874,1.74434,12.041,5048.88,A,33.21789,2459408.4263,14.0,76.0,2458990.9303,11.5,69.0
3366,TIC 270360534,Kostov 2023 arXiv:2309.14200,QuadEB,263.430502039255,43.860752149892,-11.9616,-23.9874,1.74434,12.041,5048.88,B,--,2459024.8801,9.2,--,2458989.2459,11.4,--
3367,TIC 219469945,Kostov 2022 arXiv:2202.05790,QuadEB,241.047907546962,43.0303006747434,-0.866116,-13.0913,1.08328,12.507,6322.79,A,2.717596,2458959.7065,4.1,125.0,2458961.069103,4.0,43.0
3367,TIC 219469945,Kostov 2022 arXiv:2202.05790,QuadEB,241.047907546962,43.0303006747434,-0.866116,-13.0913,1.08328,12.507,6322.79,B,14.965529,2458957.7099,6.3,80.0,2458964.924782,7.3,20.0
3368,TIC 20212631,Kostov 2023 arXiv:2309.14200,QuadEB,227.038060401645,39.97024000055,-82.8253,60.7128,5.36793,10.58,5665.4,A,0.297159,2458930.599,1.2,215.0,--,--,--
3368,TIC 20212631,Kostov 2023 arXiv:2309.14200,QuadEB,227.038060401645,39.97024000055,-82.8253,60.7128,5.36793,10.58,5665.4,B,9.47867,2458933.1418,4.4,76.0,--,--,--
3369,TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,69.9253097979945,29.1345840372292,1.09215,-1.99413,0.347484,15.116,6445.0,A,3.17353,2458818.066,5.0,103.0,2458819.6528,5.0,81.0
3369,TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,69.9253097979945,29.1345840372292,1.09215,-1.99413,0.347484,15.116,6445.0,B,13.78416,2459477.5808,7.0,41.0,2458823.6797,6.7,23.0
3370,TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,223.425163143389,52.7158479482434,2.17801,-3.59429,2.02761,12.497,5860.0,A,1.350249,2458744.7203,4.0,75.0,2458745.394749,4.0,20.0


In [71]:
eclipses = tb.Table.read("Eclipse observations.csv")
eclipses

TIC,Byear,System,Component
str13,float64,str1,str1
TIC 78568780,2022.7479,A,a
TIC 161043618,2022.3586,A,b
TIC 283940788,2022.7419,A,b
TIC 317863971,2022.8735,B,a
TIC 344541836,2022.7445,B,a
TIC 344541836,2022.8729,A,b
TIC 367448265,23.1848,B,a
TIC 375325607,22.8726,A,b
TIC 389836747,22.8675,A,b


In [7]:
tic_observations = tb.Table.read("../Process DSSI olist logs/TIC observations.csv")
tic_observations

Name,ImageNum,Gain1,Gain2,RA,Dec,PMRA,PMDec,Mag,DateTimeUtc,TIC ID,BYear,JD
str7,int32,int32,int32,float64,float64,float64,float64,float64,str19,str13,float64,float64
H300032,91,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:22:00,TIC 283940788,2022.7419373794528,2459850.8486111113
H300032,92,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:23:00,TIC 283940788,2022.7419392807785,2459850.8493055557
H300032,93,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:25:00,TIC 283940788,2022.7419430834302,2459850.8506944445
H300032,94,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:26:00,TIC 283940788,2022.7419449847562,2459850.851388889
H300032,95,300,300,8.85125,62.9015833333,-2.727,-0.154,11.73,2022-09-28 08:28:00,TIC 283940788,2022.7419487874079,2459850.8527777777
H300033,97,300,300,11.86375,64.8180277777,-2.559,-0.239,12.02,2022-09-28 08:36:00,TIC 284814380,2022.7419639980149,2459850.8583333334
H300033,98,300,300,11.86375,64.8180277777,-2.559,-0.239,12.02,2022-09-28 08:37:00,TIC 284814380,2022.7419658993408,2459850.859027778
H300033,99,300,300,11.86375,64.8180277777,-2.559,-0.239,12.02,2022-09-28 08:39:00,TIC 284814380,2022.7419697019927,2459850.8604166666
H300033,100,300,300,11.86375,64.8180277777,-2.559,-0.239,12.02,2022-09-28 08:41:00,TIC 284814380,2022.7419735046444,2459850.8618055554


In [15]:
tb.join(
    tic_observations[["TIC ID", "JD"]],
    targets[["target_name", "ephem_member", "ephem_period", "ephem_t0_a", "ephem_duration_a", "ephem_t0_b", "ephem_duration_b"]],
    keys_left="TIC ID",
    keys_right="target_name")

TIC ID,JD,target_name,ephem_member,ephem_period,ephem_t0_a,ephem_duration_a,ephem_t0_b,ephem_duration_b
str13,float64,str14,str1,float64,float64,float64,float64,float64
TIC 123098844,2459851.715277778,TIC 123098844,A,1.730707,2458685.852,--,--,--
TIC 123098844,2459851.715277778,TIC 123098844,B,11.210254,2458685.345,--,--,--
TIC 123098844,2459851.7159722224,TIC 123098844,A,1.730707,2458685.852,--,--,--
TIC 123098844,2459851.7159722224,TIC 123098844,B,11.210254,2458685.345,--,--,--
TIC 123098844,2459851.717361111,TIC 123098844,A,1.730707,2458685.852,--,--,--
TIC 123098844,2459851.717361111,TIC 123098844,B,11.210254,2458685.345,--,--,--
TIC 123098844,2459851.7138888887,TIC 123098844,A,1.730707,2458685.852,--,--,--
TIC 123098844,2459851.7138888887,TIC 123098844,B,11.210254,2458685.345,--,--,--
TIC 123098844,2459851.7131944443,TIC 123098844,A,1.730707,2458685.852,--,--,--


In [78]:
# tic = "78568780"
# member = "A"
# component = "a"
# eclipse = 2459853.0006944444

rows = []
for eclipse in eclipses:
    t = Time(eclipse["Byear"], format="byear").jd
    tic = eclipse["TIC"]
    member = eclipse["System"]
    component = eclipse["Component"]
    ephem = targets[(targets["target_name"] == tic) & (targets["ephem_member"] == member)]
    t0 = float(ephem[f"ephem_t0_{component}"])
    duration = float(ephem[f"ephem_duration_{component}"]) / 24
    period = float(ephem["ephem_period"])
    cycles = (t - t0) / period
    confirmed = abs(cycles - round(cycles)) < duration
    rows.append((tic, member, component, confirmed, period, t0, duration, cycles))

results = tb.Table(rows=rows, names=["TIC", "Member", "Component", "Confirmed", "Period", "T_0", "Duration", "Num Periods"])
results




TIC,Member,Component,Confirmed,Period,T_0,Duration,Num Periods
str13,str1,str1,bool,float64,float64,float64,float64
TIC 78568780,A,a,False,2.88838,2458468.5374,0.1625,479.3306323096533
TIC 161043618,A,b,True,1.350249,2458745.394749,0.16666666666666666,715.0109903910744
TIC 283940788,A,b,True,0.876867,2458766.180072,0.2708333333333333,1236.9662520743975
TIC 317863971,B,a,False,3.733625,2458499.5708,0.15833333333333333,374.7912636961212
TIC 344541836,B,a,True,2.755276,2458713.3358,0.20833333333333334,413.1886563358971
TIC 344541836,A,b,True,2.409932,2458715.416345,0.18333333333333335,490.9953233527232
TIC 367448265,B,a,False,1.86552,2458816.8633,0.17916666666666667,-390930.4966623794
TIC 375325607,A,b,False,1.311984,2458712.617118,0.14583333333333334,-555874.4943277219
TIC 389836747,A,b,True,2.56703,2458793.570314,0.3416666666666666,-284134.2946896902
